In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
from pathlib import Path

# Use absolute workspace path to avoid kernel working-dir issues
data_path = Path(r"D:/RAMAR TECH/FUTURE_ML_01/data/sales.csv")
print("Reading data from:", data_path)
df = pd.read_csv(data_path, encoding='latin1')
df.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [25]:
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')

monthly_sales = df.groupby(pd.Grouper(key='Order Date', freq='M'))['Sales'].sum()
monthly_sales = monthly_sales.dropna()


C:\Users\RAMAN K\AppData\Local\Temp\ipykernel_21084\2022412180.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sales = df.groupby(pd.Grouper(key='Order Date', freq='M'))['Sales'].sum()


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(monthly_sales, order=(1,1,1))
model_fit = model.fit()

# Forecast next 6 periods; ensure we end up with a Series that has a datetime index
forecast = model_fit.forecast(steps=6)  # next 6 months
if not hasattr(forecast, 'index'):
    last = monthly_sales.index[-1]
    forecast_index = pd.date_range(start=last + pd.offsets.MonthBegin(), periods=len(forecast), freq='M')
    forecast = pd.Series(forecast, index=forecast_index)
forecast


2018-01-31    75259.028080
2018-02-28    73306.799975
2018-03-31    72862.101575
2018-04-30    72760.803644
2018-05-31    72737.728970
2018-06-30    72732.472786
Freq: ME, Name: predicted_mean, dtype: float64

In [27]:
forecast


2018-01-31    75259.028080
2018-02-28    73306.799975
2018-03-31    72862.101575
2018-04-30    72760.803644
2018-05-31    72737.728970
2018-06-30    72732.472786
Freq: ME, Name: predicted_mean, dtype: float64

In [ ]:
monthly_sales.to_csv("../report/monthly_sales_summary.csv")

# Build a robust forecast DataFrame and save results
forecast_df = forecast.reset_index()
forecast_df.columns = ['Date', 'Forecasted_Sales']
forecast_df.to_csv("../report/forecast_results.csv", index=False)
